In [2]:
# UserBased Class: Same as Hüsnü Hoca's

import numpy as np
from tqdm import trange


class UserBased:
    mu: np.ndarray
    sim: np.ndarray

    def __init__(self, zero_mean: bool = True, beta: int = 1, idf: bool = False, verbosity: int = 0):
        """
        :param zero_mean:
        :param beta: Discounting parameter
        :param idf: Enable inverse document frequency management
        """
        self.zero_mean = zero_mean
        self.beta = beta
        self.idf = idf
        self.verbosity = verbosity

    def fit(self, r: np.ndarray):
        m, n = r.shape
        if self.zero_mean:
            self.mu = np.nanmean(r, axis=1)
        else:
            self.mu = np.zeros(m)

        self.sim = np.zeros((m, m))

        if self.idf:
            idf = np.log(1 + m / (~np.isnan(r)).sum(axis=0))
        else:
            idf = np.ones(n)

        if self.verbosity > 0:
            print(idf)

        for i in trange(m):
            for j in range(m):
                mask = ~np.isnan(r[i, :]) & ~np.isnan(r[j, :])

                si = r[i, mask] - self.mu[i]
                sj = r[j, mask] - self.mu[j]

                self.sim[i][j] = (si * sj * idf[mask]).sum() / (
                        np.sqrt((idf[mask] * (si ** 2)).sum()) * np.sqrt((idf[mask] * (sj ** 2)).sum()))

                total_intersection = mask.sum()

                self.sim[i][j] *= min(total_intersection, self.beta) / self.beta

        return self.sim

    def predict(self, r: np.array, u: int, top_k: int = 3) -> np.ndarray:
        """
        :param r: Rating matrix
        :param u: User u
        :param top_k: Top k neighbourhood
        :return: Calculated Rating of each item
        """

        _, n = r.shape

        score = np.zeros(n)

        for j in trange(n):
            score[j] = self.predict1(r, u, j, top_k)

        return score

    def predict1(self, r: np.array, u: int, j: int, top_k: int = 3) -> float:
        _, n = r.shape

        users_rated_j = np.nonzero(~np.isnan(r[:, j]))[0]

        topk_users = users_rated_j[self.sim[u, users_rated_j].argsort()[::-1][:top_k]]

        mean_centered_topk_user_rate = r[topk_users, j] - self.mu[topk_users]

        w = self.sim[u, topk_users]

        return np.dot(mean_centered_topk_user_rate, w) / np.abs(w).sum() + self.mu[u]

In [ ]:
# Main Code: Same as Hüsnü Hoca's

import pandas as pd
import numpy as np
from rich.console import Console

cons = Console()

df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
                 names=['user_id', 'item_id', 'rating', 'timestamp'])

r = df.pivot(index='user_id', columns='item_id', values='rating').values

irow, jcol = np.where(~np.isnan(r))

cons.print(f"{len(irow)} entries available")

idx = np.random.choice(np.arange(100_000), 1000, replace=False)
test_irow = irow[idx]
test_jcol = jcol[idx]

r_copy = r.copy()

for i in test_irow:
    for j in test_jcol:
        r_copy[i][j] = np.nan

user = UserBased(beta=3, idf=True)

sim = user.fit(r_copy)

err = []
for u, j in zip(test_irow, test_jcol):
    y_pred = user.predict1(r_copy, u, j)
    y = r[u, j]

    err.append((y_pred - y) ** 2)

cons.print(f"RMSE: {np.sqrt(np.nanmean(np.array(err)))}")

In [ ]:
# Gradient Descent

from random import random

beta_user = []
beta_item = []
# creating beta vectors for users and items with random numbers btw 0 and 1
for _ in range(r_copy.shape[0]):
  beta_user.append(random())
for _ in range(r_copy.shape[1]):
  beta_item.append(random())
# selecting the non-empty elements of r_copy
irow_copy, jcol_copy = np.where(~np.isnan(r_copy))

alpha = 0.01
for i in range(50):
  y_error_gd = 0
  for u,j in zip(irow_copy, jcol_copy):
    g_b = -(r_copy[u,j] - beta_user[u] - beta_item[j]).sum()
    y_error_gd += ((r_copy[u,j] - beta_user[u] - beta_item[j])**2)

  print(f"({i}) Gradient: [ {g_b} ], Error: [ {y_error_gd} ]")

  beta_user[u] = beta_user[u] - alpha * g_b
  beta_item[j] = beta_item[j] - alpha * g_b

In [ ]:
# Reguralization

from random import random

beta_user = []
beta_item = []

for _ in range(r_copy.shape[0]):
  beta_user.append(random())
for _ in range(r_copy.shape[1]):
  beta_item.append(random())

irow_copy, jcol_copy = np.where(~np.isnan(r_copy))

lam = random()
alpha = 0.01
for i in range(10):
  y_error_reg = 0
  for u,j in zip(irow_copy, jcol_copy):
    g_b_user = -(r_copy[u,j] - beta_user[u] - beta_item[j]).sum() - beta_user[u]*lam
    g_b_item = -(r_copy[u,j] - beta_user[u] - beta_item[j]).sum() - beta_item[j]*lam
    y_error_reg += ((r_copy[u,j] - beta_user[u] - beta_item[j])**2)

  print(f"({i}) Gradient: [ {g_b_user} , {g_b_item} ], Error: [ {y_error_reg} ]")

  beta_user[u] = beta_user[u] - alpha * g_b_user
  beta_item[j] = beta_item[j] - alpha * g_b_item

In [ ]:
# Optimization of Lambda

from random import random

lam = random()*5
alpha = 0.01
for i in range(10):
  error_lam = 0
  for u,j in zip(test_irow, test_jcol):
    g_lam = (beta_user[u]**2 + beta_item[j]**2)/2
    error_lam += ((r[u,j] - beta_user[u] - beta_item[j])**2)

  print(f"({i}) Gradient: [ {g_lam} ], Error: [ {error_lam} ]")

  lam = lam - alpha * g_lam

"""

Lambda sabit olduğu için gradient descent ile optimize edemedim (convex lambda fonksiyonunu bulamadım). 
Aklıma gelen lambdaya for loop ile 0 ve 5 arasında 0.5 artarak giden değerler vererek en iyi sonuç veren lambdayı bulmak.
Fakat manuel bir çözüm olduğu için pek hoşuma gitmedi.

"""